In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import optuna
from optuna.pruners import SuccessiveHalvingPruner
from optuna.samplers import TPESampler

from config.config_types import AppConfig
from utils.logging_utils import ExperimentLogger
from training_routine.trainer import Trainer            
from pipeline.walkforward import WFCVGenerator
from pipeline.wf_config import WFConfig
from hyperparams_search.search_utils import sample_hparams_into_cfg
#from hyperparams_search.torch_estimator import TorchFoldEstimator
#from hyperparams_search.randomsearch import RandomSearch

from utils.gpu_test import gpu_test
from utils.paths import CONFIG_DIR
from utils.custom_formatter import setup_logger






from utils.logging_utils import ExperimentLogger
from training_routine.trainer import Trainer            
from pipeline.walkforward import WFCVGenerator
from pipeline.wf_config import WFConfig
from utils.gpu_test import gpu_test
from utils.paths import CONFIG_DIR
from utils.custom_formatter import setup_logger

from models import create_model 



/home/corte911/.conda/envs/torch_h200_py312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

# setup logger
console_logger = setup_logger("Experiment", level="INFO")

# --- GPU check (PyTorch) ---
gpu_test()
console_logger.info("GPU check complete.")

# -------- load config --------
cfg = AppConfig.from_dict(f"{CONFIG_DIR}/random_debug.yaml")
print(cfg)

print(cfg.model)

12:02:27 - Experiment - INFO - GPU check complete. (1149415777.py:6)


AppConfig(model=ModelConfig(name='mlp', hparams={'output_activation': 'linear'}, search={'dropout_rate': FloatSpec(low=0.0, high=0.5, log=False, type='float'), 'activation': CatSpec(choices=['relu', 'gelu', 'tanh'], type='cat'), 'n_layers': IntSpec(low=1, high=10, type='int'), 'n_hidden': IntSpec(low=64, high=1024, type='int')}), trainer=TrainerConfig(hparams={'epochs': 50, 'batch_size': 512, 'metrics': ['mae', 'mse', 'dir_acc'], 'loss': 'mse', 'val_every': 1}, search={'lr': FloatSpec(low=1e-05, high=0.01, log=True, type='float'), 'weight_decay': FloatSpec(low=0, high=0.01, log=False, type='float'), 'batch_size': IntSpec(low=32, high=256, type='int')}), walkforward=WFConfig(ratio_train=3, ratio_val=1, ratio_test=1, step=251, lags=20, max_folds=None), experiment=ExperimentConfig(name='mlp_random_search', hyperparams_search=True, monitor='val_loss', mode='min', type='price_prediction', n_trials=25, random_state=42), data={'target_col': 'y', 'feature_cols': ['feature_0', 'feature_1', 'fea

In [ ]:


# -------- data + components --------
#logger = ExperimentLogger(cfg)


wf = WFCVGenerator(config=cfg.walkforward)

# instantiate trainer (PyTorch)
#trainer = Trainer(cfg, logger)

# model input size: number of lags (columns are constant across folds)
input_shape = cfg.walkforward.lags            # int is fine; build_model handles it
max_folds = cfg.walkforward.max_folds

if cfg.model.name.lower() == "cnn1d":
    input_shape = (1, cfg.walkforward.lags)  # (C, L)
elif cfg.model.name.lower() == "mlp":
    input_shape = (cfg.walkforward.lags,)
else:
    console_logger.warning(f"Model: {cfg.model.name} not recognized!")

# Get bool for search
hyperparams_search = cfg.experiment.hyperparams_search


# -------- train per fold --------
for fold, data in enumerate(wf.folds()):
    if max_folds is not None and fold >= max_folds:
        break  # allow running subset of folds

    if hyperparams_search:
        # ---- pick a single fold 'data' and run Optuna on it ----
        direction = "minimize" if cfg.experiment.mode.lower() == "min" else "maximize"
        n_trials  = getattr(cfg.experiment, "n_trials", 50)
        n_jobs    = 1  # for a single fold on one machine; raise if you parallelize

        # Compute (fixed) input shape for this base cfg; it won't change with hparams
        def make_input_shape(c):
            return (c.walkforward.lags, )

        input_shape = make_input_shape(cfg)

        # ---- define Optuna objective for this *one fold* ----
        def objective(trial: optuna.trial.Trial) -> float:
            # 1) sample hparams → a NEW cfg (dict or dataclass, depending on your function)

            trial_cfg = sample_hparams_into_cfg(cfg, trial)  # returns same "type" you pass in

            trial_cfg = AppConfig.from_dict(trial_cfg)

            console_logger.critical(trial_cfg.model)
            #console_logger.critical(trial_cfg.trainer)


            # 2) fresh trainer per trial (avoid any state carry-over)
            trial_logger = ExperimentLogger(trial_cfg)
            trial_logger.begin_trial(trial.number)

            trial_trainer = Trainer(trial_cfg, trial_logger)

            # 3) (re)compute input shape from the *trial* cfg if model family could change
            shp = make_input_shape(trial_cfg)

            # 4) build a fresh model for this trial
            model = create_model(trial_cfg.model, shp)

            # 5) epoch-wise reporting so ASHA can prune early
            def report_cb(epoch: int, val_metric: float):
                trial.report(val_metric, step=epoch)   # strictly increasing step
                return trial.should_prune()

            # 6) fit/eval only on THIS fold’s (train,val). Your `data` tuple already contains them.
            val_metric = trial_trainer.fit_eval_fold(
            #    model, data, trial=trial.number, fold=fold, report_cb=report_cb
                model, data, trial=trial.number, fold=fold
            )

            # 7) return the scalar according to direction
            return val_metric  # study direction handles min/max

        # ---- build study (TPE + ASHA) and run for this fold ----
        study = optuna.create_study(
            direction=direction,
            sampler=TPESampler(seed=cfg.experiment.random_state, multivariate=True),
            pruner=SuccessiveHalvingPruner(min_resource=3, reduction_factor=3),  # ASHA-like pruning
        )
        study.optimize(objective, n_trials=n_trials, n_jobs=n_jobs)

        # ---- log & (optionally) retrain best-once for the fold ----
        console_logger.info(f"[Fold {fold}] Best value:  {study.best_value:.6f}")
        console_logger.info(f"[Fold {fold}] Best params: {study.best_params}")

        # Optional: rebuild a cfg from best params and do a final “full” run (e.g., full epochs)
        best_cfg = sample_hparams_into_cfg(cfg, optuna.trial.FixedTrial(study.best_params))
        best_logger  = ExperimentLogger(best_cfg)
        best_trainer = Trainer(best_cfg, best_logger)
        best_model   = create_model(best_cfg.model, input_shape)

        # If you keep shorter epochs during search, you can override here:
        # best_cfg.trainer.params["epochs"] = cfg.trainer.params["epochs"]  # full epochs

        _ = best_trainer.fit_eval_fold(best_model, data, trial=-1, fold=fold, report_cb=None)

        # --- log best results ---
        #console_logger.info(f"[Fold {fold}] Best params: {result['best_params']}")
        #console_logger.info(f"[Fold {fold}] Best {cfg.experiment.monitor}: {result['best_selection_score']:.6f}")
        


    else:
        # Keep model creation inside the loop to avoid weight leakage across folds
        model = create_model(cfg.model, input_shape)       

        if fold == 0:
            console_logger.critical(f"model: {model}")

        trainer.fit_eval_fold(model, data, trial=0, fold=fold)

console_logger.warning("Training completed!")



/home/corte911/.conda/envs/torch_h200_py312/lib/python3.12/site-packages/optuna/_experimental.py:32: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-10-18 12:05:12,938] A new study created in memory with name: no-name-9aee4c37-cc5c-40ca-948e-3e60bba53900
12:05:12 - Search_utils - DEBUG - model_keys: ['dropout_rate', 'activation', 'n_layers', 'n_hidden', 'output_activation'] (search_utils.py:66)
12:05:12 - Search_utils - DEBUG - cfg["model"]: {'name': 'mlp', 'hparams': {'output_activation': 'linear'}, 'search': {'dropout_rate': {'low': 0.0, 'high': 0.5, 'log': False, 'type': 'float'}, 'activation': {'choices': ['relu', 'gelu', 'tanh'], 'type': 'cat'}, 'n_layers': {'low': 1, 'high': 10, 'type': 'int'}, 'n_hidden': {'low': 64, 'high': 1024, 'type': 'int'}}} (search_utils.py:67)
12:05:12 - Search_utils - DEBUG - cfg["model"] after the cycle: {'name': 'mlp', 'hparams': {'output_activation': 'linear'}

TypeError: 'TrainerConfig' object is not subscriptable